In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = mls
band = [6]
commitnumber = de5d43e
molecule = {'co2': 0.0004}
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band4
  atmpro = mls
  band = 4
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [1, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 980
  vmin = 800
  w_diffuse = [(1.75,), (1.66, 1.9)]
  wgt = [(0.75,), (0.75, 0.95)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.000758                  NaN       NaN
0.000312      NaN       NaN                  1.0 -0.000777
0.000750      2.0 -0.000760                  2.0 -0.000801
0.001052      3.0 -0.000764                  3.0 -0.000779
0.001476      4.0 -0.000768                  4.0 -0.000782
0.002070      5.0 -0.000772                  5.0 -0.000788
0.002904      6.0 -0.000775                  6.0 -0.000793
0.004074      7.0 -0.000777                  7.0 -0.000797
0.005714      8.0 -0.000779                  8.0 -0.000801
0.008015      9.0 -0.000781                  9.0 -0.000804
0.011243     10.0 -0.000781                 10.0 -0.000809
0.015771     11.0 -0.000781                 11.0 -0.000817
0.022122     12.0 -0.000781                 12.0 -0.000823
0.031031     13.0 -0.000780                 13.0 -0.000828
0.043528     14.0 -0.000777                 14.0 -0.000832
0.061057     15.0 -0.000765                 15.0 -0.000839
0.085645     16.0 -0.000680                 16.0 -0.000822
0.120136     17.0 -0.000381                 17.0 -0.000682
0.168516     18.0  0.000260                 18.0 -0.000318
0.236378     19.0  0.001425                 19.0  0.000388
0.331549     20.0  0.003374                 20.0  0.001597
0.465100     21.0  0.006455                 21.0  0.003557
0.652400     22.0  0.011099                 22.0  0.006797
0.915100     23.0  0.016166                 23.0  0.010429
1.283650     24.0  0.016278                 24.0  0.010644
1.800600     25.0  0.011654                 25.0  0.007673
2.525700     26.0  0.007232                 26.0  0.004759
3.542800     27.0  0.004017                 27.0  0.002575
4.969550     28.0  0.001869                 28.0  0.001080
6.970850     29.0  0.000534                 29.0  0.000117
9.778100     30.0 -0.000275                 30.0 -0.000491
13.715850    31.0 -0.000768                 31.0 -0.000878
19.239350    32.0 -0.001044                 32.0 -0.001101
26.987250    33.0 -0.001168                 33.0 -0.001208
37.855300    34.0 -0.001224                 34.0 -0.001263
53.100050    35.0 -0.001257                 35.0 -0.001295
73.887500    36.0 -0.001279                 36.0 -0.001313
97.662500    37.0 -0.001307                 37.0 -0.001337
121.437500   38.0 -0.001347                 38.0 -0.001355
145.212500   39.0 -0.001385                 39.0 -0.001339
168.987500   40.0 -0.001436                 40.0 -0.001362
192.762500   41.0 -0.001530                 41.0 -0.001416
216.537500   42.0 -0.001629                 42.0 -0.001547
240.312500   43.0 -0.001674                 43.0 -0.001621
264.087500   44.0 -0.001654                 44.0 -0.001620
287.862500   45.0 -0.001563                 45.0 -0.001545
311.637500   46.0 -0.001398                 46.0 -0.001402
335.412500   47.0 -0.001159                 47.0 -0.001169
359.187500   48.0 -0.000842                 48.0 -0.000872
382.962500   49.0 -0.000448                 49.0 -0.000505
406.737500   50.0  0.000021                 50.0 -0.000061
430.512500   51.0  0.000561                 51.0  0.000452
454.287500   52.0  0.001166                 52.0  0.001030
478.062500   53.0  0.001832                 53.0  0.001675
501.837500   54.0  0.002554                 54.0  0.002388
525.612500   55.0  0.003334                 55.0  0.003131
549.387500   56.0  0.004172                 56.0  0.003940
573.162500   57.0  0.005070                 57.0  0.004818
596.937500   58.0  0.006028                 58.0  0.005768
620.712500   59.0  0.007045                 59.0  0.006792
644.487500   60.0  0.008123                 60.0  0.007857
668.262500   61.0  0.009258                 61.0  0.008964
692.037500   62.0  0.010452                 62.0  0.010138
715.812500   63.0  0.011704                 63.0  0.011378
739.587500   64.0  0.013011                 64.0  0.012684
76

# Fluxes by Layer

CRD                              CLIRAD  \
                         flug          fldg      fnetg       flug   
pressure     level                                                  
0.000000e+00 1     -61.259575  0.000000e+00 -61.259575        NaN   
1.000000e-08 1            NaN           NaN        NaN -61.296166   
6.244000e-04 2     -61.259575  3.165274e-08 -61.259575 -61.296166   
8.759000e-04 3     -61.259575  4.462509e-08 -61.259575 -61.296166   
1.228600e-03 4     -61.259575  6.350668e-08 -61.259575 -61.296166   
1.723400e-03 5     -61.259575  9.105809e-08 -61.259575 -61.296166   
2.417400e-03 6     -61.259575  1.312444e-07 -61.259575 -61.296166   
3.390900e-03 7     -61.259575  1.898588e-07 -61.259575 -61.296166   
4.756500e-03 8     -61.259575  2.753433e-07 -61.259575 -61.296166   
6.672000e-03 9     -61.259576  3.999923e-07 -61.259575 -61.296166   
9.358900e-03 10    -61.259576  5.817322e-07 -61.259575 -61.296167   
1.312780e-02 11    -61.259577  8.466751e-07 -61.259576 -61.296167   
1.841450e-02 12    -61.259577  1.232869e-06 -61.259576 -61.296168   
2.583020e-02 13    -61.259579  1.795735e-06 -61.259577 -61.296169   
3.623230e-02 14    -61.259580  2.616023e-06 -61.259578 -61.296171   
5.082340e-02 15    -61.259583  3.814291e-06 -61.259579 -61.296173   
7.129060e-02 16    -61.259587  5.706647e-06 -61.259581 -61.296177   
1.000000e-01 17    -61.259593  9.330929e-06 -61.259583 -61.296182   
1.402710e-01 18    -61.259602  1.712085e-05 -61.259585 -61.296192   
1.967600e-01 19    -61.259617  3.375540e-05 -61.259583 -61.296206   
2.759970e-01 20    -61.259639  6.879693e-05 -61.259570 -61.296228   
3.871000e-01 21    -61.259667  1.414804e-04 -61.259526 -61.296258   
5.431000e-01 22    -61.259696  2.900180e-04 -61.259406 -61.296297   
7.617000e-01 23    -61.259706  5.873638e-04 -61.259119 -61.296340   
1.068500e+00 24    -61.259661  1.129596e-03 -61.258531 -61.296369   
1.498800e+00 25    -61.259583  1.881588e-03 -61.257702 -61.296405   
2.102400e+00 26    -61.259560  2.691593e-03 -61.256869 -61.296509   
2.949000e+00 27    -61.259641  3.497292e-03 -61.256143 -61.296718   
4.136600e+00 28    -61.259859  4.280543e-03 -61.255578 -61.297059   
5.802500e+00 29    -61.260249  5.039782e-03 -61.255209 -61.297564   
8.139200e+00 30    -61.260852  5.790202e-03 -61.255061 -61.298270   
1.141700e+01 31    -61.261720  6.551462e-03 -61.255168 -61.299227   
1.601470e+01 32    -61.262920  7.332933e-03 -61.255587 -61.300498   
2.246400e+01 33    -61.264531  8.147146e-03 -61.256384 -61.302159   
3.151050e+01 34    -61.266689  9.053029e-03 -61.257636 -61.304346   
4.420010e+01 35    -61.269615  1.013920e-02 -61.259475 -61.307285   
6.200000e+01 36    -61.273588  1.146158e-02 -61.262127 -61.311265   
8.577500e+01 37    -61.278742  1.301461e-02 -61.265727 -61.316433   
1.095500e+02 38    -61.283905  1.449634e-02 -61.269408 -61.321606   
1.333250e+02 39    -61.289207  1.600444e-02 -61.273203 -61.326842   
1.571000e+02 40    -61.294640  1.753582e-02 -61.277104 -61.332013   
1.808750e+02 41    -61.300310  1.916111e-02 -61.281149 -61.337320   
2.046500e+02 42    -61.306621  2.116450e-02 -61.285456 -61.343070   
2.284250e+02 43    -61.313950  2.390651e-02 -61.290043 -61.349869   
2.522000e+02 44    -61.322352  2.759304e-02 -61.294759 -61.357743   
2.759750e+02 45    -61.331793  3.237569e-02 -61.299417 -61.366604   
2.997500e+02 46    -61.342220  3.840145e-02 -61.303818 -61.376374   
3.235250e+02 47    -61.353572  4.581506e-02 -61.307757 -61.387017   
3.473000e+02 48    -61.365778  5.475828e-02 -61.311020 -61.398319   
3.710750e+02 49    -61.378760  6.536854e-02 -61.313391 -61.410271   
3.948500e+02 50    -61.392431  7.777697e-02 -61.314654 -61.422870   
4.186250e+02 51    -61.406694  9.209873e-02 -61.314595 -61.435987   
4.424000e+02 52    -61.421451  1.084366e-01 -61.313015 -61.449445   
4.661750e+02 53    -61.436606  1.268759e-01 -61.309730 -61.463228   
4.899500e+02 54    -61.452059  1.474878e-01 -61.304572 -61.477295   
5.137250e+02 55    -61.4

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')